In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import Bounds
from calibration.solver_calibration import Calibration

from models.vasicek import Vasicek
from models.cir import CIR

In [2]:
inital_model_params = {
    "kappa": 0.5,
    "mu_r": 0.03,
    "sigma": 0.03,
    "mu": 0,
    "gamma": 0.01,
    "h": 10
}

In [3]:
yield_curve_full = pd.read_csv("data/yield_curve02132021.csv")
yield_curve = pd.DataFrame(yield_curve_full.iloc[0, 1:]/100).astype(float)
yield_curve["Maturity"] = list(range(1, len(yield_curve) + 1))
yield_curve.columns = ["Yield", "Maturity"]
yield_curve["Cum. Yield"] = yield_curve["Yield"] * yield_curve["Maturity"]
yield_curve["Price"] = np.exp(-yield_curve["Cum. Yield"])

In [4]:
prices = yield_curve["Price"].values
Ts = yield_curve["Maturity"].values

In [5]:
prices = prices.reshape((1, 30))

In [26]:
vasicek = Vasicek(inital_model_params)
optimize_args = ("kappa", "mu_r", "sigma")
bounds = Bounds([0.001, 0.001, 0.01], [1, 0.2, 0.2])

In [27]:
if __name__ == "__main__":
    calibrator = Calibration(
        model_class=Vasicek, n=100, m=26, r0=0.001,
        model_params=inital_model_params, optimize_args=optimize_args)
    optimal_vas = calibrator.calibrate(
        initial_values=(inital_model_params["kappa"], inital_model_params["mu_r"], inital_model_params["sigma"]),
        Ts=Ts,
        prices=prices,
        bounds=bounds
    )

In [28]:
optimal_vas

      fun: 0.03581538717435763
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-8.47932836e-06, -3.89625832e-05,  1.25352222e+00])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 216
      nit: 34
     njev: 54
   status: 0
  success: True
        x: array([0.09379583, 0.03578758, 0.01      ])

In [6]:
cir = CIR(inital_model_params)
optimize_args = ("kappa", "mu_r", "sigma")
bounds = Bounds([0.001, 0.001, 0.01], [1, 0.2, 0.2])

In [7]:
if __name__ == "__main__":
    calibrator = Calibration(
        model_class=CIR, n=100, m=26, r0=0.001,
        model_params=inital_model_params, optimize_args=optimize_args)
    optimal_cir = calibrator.calibrate(
        initial_values=(inital_model_params["kappa"], inital_model_params["mu_r"], inital_model_params["sigma"]),
        Ts=Ts,
        prices=prices,
        bounds=bounds
    )

In [8]:
optimal_cir

      fun: 0.05727216988360956
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([14.70943109, 17.70598098, 18.44034574])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 564
      nit: 36
     njev: 141
   status: 0
  success: True
        x: array([0.07667018, 0.04420769, 0.11789453])